# Python Session 5: Introduction to Recurrent neural networks

# RNN

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
import torch.optim as optim

import matplotlib.pyplot as plt

## 5.1. Data preparation

In [2]:
'''
X
'''

np.random.seed(2022)
nYear=11
sample_size=150000
param_beta = np.array([-3.0, 1.0, 2.5])
p = len(param_beta) # number of explanatory variable including 1
para = pd.Series({'beta': param_beta, 'phi': 0.9, 'sig': 0.5})

X1 = np.stack((np.repeat(1, sample_size),
              np.random.binomial(n=1, p=0.5, size=sample_size),
              np.random.binomial(n=1, p=0.5, size=sample_size)),
              axis=1)

X0 = X1[:,1:] #not used
X_sim = np.stack([X1]*nYear, axis=1) #not used
X_sim.shape #not used

(150000, 11, 3)

In [3]:
lamb = np.exp(np.matmul(X_sim, para['beta'][:, np.newaxis]))
print('lamb shape', lamb.shape)


lamb shape (150000, 11, 1)


In [ ]:
'''
lambda, R, n
'''


sig = para['sig']
phi = para['phi']
n = np.full((sample_size, nYear), 999)
Rhat = np.full((sample_size, nYear), 0.0)
c_mean=-sig/np.sqrt(1-phi**2)/2
for i in range(sample_size):
    for t in range(nYear):
        if t==0:
            Rhat[i,t] = np.random.normal(c_mean, sig/np.sqrt(1-phi**2), size=1) #= phi*R0[i] + epsilon[i,1]
        else:
            Rhat[i,t] = np.random.normal((Rhat[i,t-1]-c_mean)*phi, sig, size=1) +c_mean
            
        n[i,t] = np.random.poisson(lam=lamb[i,t]*np.exp(Rhat[i,t]), size=1) #rpois(1, mu_n )



N_sim=n
lamb_sim = lamb

print('N_sim shape', N_sim.shape)
print('lamb_sim shape', lamb_sim.shape) # lamb 복제 (nYear만큼)

# You are given the following data

1. N_sim: claim history of size [sample_size, 11] 
2. lamb_sim: true priori rate of size [sample_size, 11, 1]
3. X_sim: explanatory variables of size [sample_size, 11, 3]

In [ ]:
print(N_sim.shape, lamb_sim.shape, X_sim.shape)

In [ ]:
np.unique(lamb_sim)

## Preprocessing

## Our goal is to make the following data:

y_future_train:  T2~T11 [N1, T, 1]

lamb_future_train:  T2~T11 [N1, T, 1]

lamb_current_train: T1~T10 [N1, T, 1]

y_future_valid:  T2~T11 [N2, T, 1]

lamb_future_valid:  T2~T11 [N2, T, 1]

lamb_current_valid: T1~T10 [N2, T, 1]

y_future_test:  T2~T11 [N3, T, 1]

lamb_future_test:  T2~T11 [N3, T, 1]

lamb_current_test: T1~T10 [N3, T, 1]




In [ ]:
sample_size

150000

In [ ]:
Y = torch.tensor(N_sim[:,:,np.newaxis ], dtype=torch.float32) #shape: [none, 11, 1], #time t1~t11
LAMB = torch.tensor(lamb_sim, dtype=torch.float32)

N_train = int(sample_size*0.5)
N_valid = int(sample_size*0.6)
N_test  = int(sample_size*1.0)

y_future_train, lamb_future_train, y_current_train, lamb_current_train =  Y[:N_train, 1:, :], LAMB[:N_train, 1:, :], Y[:N_train,:-1, :], LAMB[:N_train,:-1, :]
y_future_valid, lamb_future_valid, y_current_valid, lamb_current_valid =  Y[N_train:N_valid, 1:, :], LAMB[N_train:N_valid, 1:, :], Y[N_train:N_valid,:-1, :], LAMB[N_train:N_valid,:-1, :]
y_future_test, lamb_future_test, y_current_test, lamb_current_test =  Y[N_valid:, 1:, :], LAMB[N_valid:, 1:, :], Y[N_valid:,:-1, :], LAMB[N_valid:,:-1, :]


In [ ]:
x_current_train = torch.concat([y_current_train, lamb_current_train], axis=2)
x_current_valid = torch.concat([y_current_valid, lamb_current_valid], axis=2)
x_current_test = torch.concat([y_current_test, lamb_current_test], axis=2)

In [ ]:
x_current_test.shape

torch.Size([60000, 10, 2])

## Model definition

In [ ]:
train = torch.utils.data.TensorDataset(x_current_train, lamb_future_train, y_future_train)
test = torch.utils.data.TensorDataset(x_current_test, lamb_future_test, y_future_test)

batch_size = 32
train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test, batch_size=batch_size, shuffle=True)